# Collecting data on decentralized exchanges using BitQuery GraphQL API

**First we define how to run a query using API key from BitQuery**

In [1]:
import requests
import pandas as pd
import yaml


with open('api.yaml', 'r') as f:
    parsed_yaml = yaml.load(f, Loader=yaml.FullLoader)
    API_KEY = parsed_yaml['api']


def run_query(query):  # A simple function to use requests.post to make the API call.
    headers = {'X-API-KEY': API_KEY}
    request = requests.post('https://graphql.bitquery.io/', json={'query': query}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed and return code is {}.      {}'.format(request.status_code, query))


**The below query fetches the top decentralized exchanges based on their protocol and sorted on the number of trades**

In [2]:
query = """
{
  ethereum {
    dexTrades(options: {limit: 100, desc: "count"}) {
      count
      protocol
    }
  }
}
"""
result = run_query(query)  # Execute the query
print ('Result - {}'.format(result))

Result - {'data': {'ethereum': {'dexTrades': [{'count': 87766950, 'protocol': 'Uniswap v2'}, {'count': 14236600, 'protocol': 'Uniswap v3'}, {'count': 5911016, 'protocol': 'IDEX'}, {'count': 3736126, 'protocol': 'EtherDelta'}, {'count': 2604842, 'protocol': 'Balancer Pool Token'}, {'count': 2538335, 'protocol': 'Uniswap'}, {'count': 2020822, 'protocol': 'Zerox Exchange v2'}, {'count': 1046443, 'protocol': 'Bancor Network'}, {'count': 1023483, 'protocol': 'Kyber Network v3'}, {'count': 868355, 'protocol': 'Bancor Network v2'}, {'count': 768626, 'protocol': 'dYdX2'}, {'count': 743531, 'protocol': 'Zerox Exchange'}, {'count': 487296, 'protocol': 'Matching Market'}, {'count': 466653, 'protocol': 'Zerox Exchange v3'}, {'count': 328797, 'protocol': 'Curve'}, {'count': 328141, 'protocol': 'Kyber Network v4'}, {'count': 322422, 'protocol': 'Zerox Exchange v4'}, {'count': 320733, 'protocol': 'Token.Store'}, {'count': 279656, 'protocol': 'Mooniswap'}, {'count': 234004, 'protocol': 'One Inch Liqui

In [3]:
# Convert the result to a dataframe
df = pd.json_normalize(result['data']['ethereum']['dexTrades'])
print(df)

       count                 protocol
0   87766950               Uniswap v2
1   14236600               Uniswap v3
2    5911016                     IDEX
3    3736126               EtherDelta
4    2604842      Balancer Pool Token
5    2538335                  Uniswap
6    2020822        Zerox Exchange v2
7    1046443           Bancor Network
8    1023483         Kyber Network v3
9     868355        Bancor Network v2
10    768626                    dYdX2
11    743531           Zerox Exchange
12    487296          Matching Market
13    466653        Zerox Exchange v3
14    328797                    Curve
15    328141         Kyber Network v4
16    322422        Zerox Exchange v4
17    320733              Token.Store
18    279656                Mooniswap
19    234004  One Inch Liquidity Pool
20    181219                     Dodo
21     91447         Kyber Network v2
22     77242          DDEX Hydro v1.1
23     73696          DDEX Hydro v1.0
24     48304         AirSwap Exchange
25     31764

**This is a good place to start as it shows us the most popular decentralized exchanges we can now compare**

**The below query gives the daily trade volume of USDT and ETH on Uniswap**

In [ ]:
query = """
{
  ethereum(network: ethereum) {
    dexTrades(options: {limit: 100, asc: "timeInterval.day"},
      date: {since:"2022-05-05"}
      exchangeName: {is: "Uniswap"},
      baseCurrency: {is: "0xdac17f958d2ee523a2206206994597c13d831ec7"},
      quoteCurrency: {is: "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2"}) {


      timeInterval {
        day(count: 1)
      }

      baseCurrency {
        symbol
        address
      }
      baseAmount
      quoteCurrency {
        symbol
        address
      }
      quoteAmount

      trades: count
      quotePrice

      maximum_price: quotePrice(calculate: maximum)
      minimum_price: quotePrice(calculate: minimum)

      open_price: minimum(of: block get: quote_price)
      close_price: maximum(of: block get: quote_price)
  		tradeAmount(in: USDT)
    }
  }
}
"""
result = run_query(query)  # Execute the query
print ('Result - {}'.format(result))

In [ ]:
# Convert the result to a dataframe
df = pd.json_normalize(result['data']['ethereum']['dexTrades'])
print(df)

In [11]:
# Convert the result to a dataframe
df = pd.json_normalize(result['data']['ethereum']['dexTrades'])
print(df)

      baseAmount    quoteAmount  trades  quotePrice  maximum_price  \
0   1.362603e+08   48875.224526    4405    0.000359       0.000400   
1   1.883495e+08   70232.189943    4773    0.000373       0.000399   
2   1.089553e+08   40989.540984    3823    0.000376       0.000390   
3   2.604093e+08  102702.544828    5721    0.000394       0.000450   
4   2.669112e+08  113164.877628    7777    0.000424       0.000507   
5   2.471876e+08  105352.182036    7371    0.000426       0.000749   
6   4.091710e+08  182431.703348    8218    0.000446       0.000986   
7   6.580154e+08  342469.695267    9021    0.000520       0.057808   
8   2.239275e+08  108000.309110    6357    0.000482       0.000523   
9   1.229483e+08   61628.607490    5726    0.000501       0.000856   
10  3.310531e+07   16211.295075    2102    0.000490       0.000500   

    minimum_price              open_price             close_price tradeAmount  \
0        0.000338   0.0003418299240166425  0.00036359136928855326        None 